In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
import matplotlib.pyplot as plt


In [14]:
# train 데이터 로드
with open('/Users/soyoung/ml_project/emotion_proj/1/emo_train_dataset_real.pkl', 'rb') as f:
    train_data = pickle.load(f)

# test 데이터 로드
with open('/Users/soyoung/ml_project/emotion_proj/1/emo_test_dataset_real.pkl', 'rb') as f:
    test_data = pickle.load(f)

train_data = np.array(train_data)
test_data = np.array(test_data)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (28709, 2) + inhomogeneous part.

In [6]:


# 데이터 확인
print(type(train_data))
print(len(train_data))
print(type(train_data[0]))
print(train_data[0].shape)

<class 'list'>
28709
<class 'tuple'>


AttributeError: 'tuple' object has no attribute 'shape'

In [10]:
class EmotionDataset(Dataset):
    def __init__(self, data):
        self.data = data['data']
        self.labels = data['label']
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # 이미지를 0~1로 정규화합니다.
        image = torch.FloatTensor(self.data[index]) / 255.
        # 이미지와 라벨을 반환합니다.
        return image.unsqueeze(0), self.labels[index]


In [11]:
# 데이터셋 생성
train_dataset = EmotionDataset(train_data)
test_dataset = EmotionDataset(test_data)

# 데이터 로더 생성
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


TypeError: list indices must be integers or slices, not str

In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, 7)  # 7개의 감정 라벨
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # Convolutional Layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Flatten
        x = x.view(-1, 256 * 6 * 6)
        
        # Fully Connected Layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        
        return x



In [ ]:

# CNN 모델이 정의되었으므로, 모델을 인스턴스화하고 손실 함수와 최적화 알고리즘을 설정합니다.

# 모델 생성
model = EmotionCNN()

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def train(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward Pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        # Backward Pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)
    
    return epoch_loss, epoch_acc

def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward Pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)
    
    return epoch_loss, epoch_acc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 학습
num_epochs = 50
train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_dataloader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_dataloader, criterion)
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# 테스트
test_loss, test_acc = evaluate(model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')



In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].plot(train_losses, label='Train')
axes[0].plot(val_losses, label='Val')
axes[0].set_title('Loss')
axes[0].legend()

axes[1].plot(train_accs, label='Train')
axes[1].plot(val_accs, label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()

plt.show()


In [ ]:
# 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/emotion_proj/model/emotion_cnn.pt')


In [ ]:
import numpy as np

model.eval()

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        inputs = inputs.cpu().numpy().transpose((0, 2, 3, 1))
        labels = labels.cpu().numpy()
        preds = preds.cpu().numpy()
        
        for i in range(len(inputs)):
            input_ = inputs[i]
            label_ = labels[i]
            pred_ = preds[i]
            
            # 이미지 출력
            plt.imshow(input_)
            plt.title(f'True: {label_to_emotion[label_]}, Pred: {label_to_emotion[pred_]}')
            plt.axis('off')
            plt.show()
